In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint

import joblib
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Input
from keras.backend import clear_session
from keras.optimizers import Adam

In [91]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [92]:
def dl_acc_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['accuracy'], label='train_err', marker = '.')
    plt.plot(history['val_accuracy'], label='val_err', marker = '.')

    plt.ylabel('Acc')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [93]:
data_train = pd.read_csv('../2024.10.16_미니프로젝트 3차_실습자료\data01_train.csv')
data_test = pd.read_csv('../2024.10.16_미니프로젝트 3차_실습자료\data01_test.csv')
data_is_dy_train = joblib.load('../2024.10.16_미니프로젝트 3차_실습자료\Is_Dynamic_DF.pkl')
feature = pd.read_csv('../2024.10.16_미니프로젝트 3차_실습자료/features.csv')
data_test.drop(columns='subject',inplace=True)
data_train.drop(columns='subject',inplace=True)
data_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


# x,y 분리

In [94]:
target = 'Activity'

x = data_is_dy_train.drop(columns=target)
x_columns = x.columns
y = data_is_dy_train[target]
x.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.032207,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.246705,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,0.388765,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,0.050888,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.013902,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758


In [95]:
def scaled_to_PCA(x_data:pd.DataFrame,n_components:int=20)->pd.DataFrame:
    # 정규화
    scaler = StandardScaler()
    x_columns = x_data.columns
    x = scaler.fit_transform(x_data)
    x = pd.DataFrame(x,columns=x_columns)
    
    # PCA Decomposition
    pca = PCA(n_components=n_components)
    principalComponents = pca.fit_transform(x)
    principalDF = pd.DataFrame(data=principalComponents,columns=['Comp']*n_components)
    return principalDF

In [96]:
x = scaled_to_PCA(x)
x.head()

,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp
0,-13.457865,-1.018722,-1.473502,-3.173522,-3.552740,1.630544,-0.888701,0.713501,-0.270988,0.813511,-0.021769,1.540031,-1.915225,-0.722601,0.087224,1.801308,-0.700502,-0.810181,-0.112014,-0.595766
1,-15.708905,5.066815,0.556654,1.849810,3.893717,0.086818,-1.068139,1.563345,-2.994460,1.016738,-1.870366,3.125093,0.417878,-1.651049,-0.324364,1.136956,0.561593,0.051177,1.144562,0.589228
2,-15.439979,9.205352,-5.061634,-2.351507,0.204771,-2.022815,2.227008,-0.607504,-3.044941,-0.123431,-0.317540,-0.744291,1.405598,1.239665,0.472004,0.394798,-0.978776,-1.820907,0.728341,0.479334
3,14.589192,-1.414871,-5.251271,-0.649552,1.798735,2.422224,-0.882786,0.045963,2.826131,2.016632,2.075050,-0.667751,0.945008,2.055222,-1.899713,-0.506332,-1.566108,-2.561922,-1.828632,0.286699
4,13.079971,-8.621200,-1.769857,4.143246,-4.135092,-1.148985,2.765234,4.072062,-2.504126,0.714006,-3.970046,-1.836319,-0.099330,3.250702,1.025094,4.143225,3.571120,0.474742,-1.844412,1.960869


# train_test_split

In [97]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2,stratify=y,random_state=1)
x_train

,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp,Comp
274,18.454875,-0.871522,-4.932407,0.748591,2.710184,1.380837,-6.739020,-3.808989,-1.357661,0.275442,-0.777903,-0.871714,2.242044,0.521593,2.616491,2.418288,-4.734902,1.029414,-2.477173,-0.331508
1088,19.547842,-9.405171,1.468663,-0.200240,-1.043843,-7.280280,-8.351449,4.131062,-6.252021,-2.510237,-1.071021,-8.702470,2.919240,-1.426429,-0.381289,1.023268,2.984803,1.898377,1.151881,-0.309476
1574,25.733168,0.336262,-3.287559,7.522264,-2.786794,-3.195890,0.687250,3.285219,0.431374,-1.514552,-4.195558,4.173144,-1.698928,-1.983247,-1.079221,-1.486309,0.020943,-0.377837,1.624556,-1.407891
3962,15.591445,-4.122439,-3.535831,2.247361,-0.380019,0.526445,-0.844063,-1.918809,-2.670957,-2.055755,0.635450,3.067931,-0.476823,2.174081,0.333023,0.608249,-0.644112,-1.345315,0.166784,-0.342600
5805,21.296862,0.045979,-2.812978,1.812215,0.966256,3.878693,0.211047,-3.625568,-6.038812,-0.814226,3.478152,1.704580,-4.106288,0.059758,-1.564833,-4.305293,-0.445909,0.316853,-0.699786,-2.340072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,16.457141,2.348713,-2.664471,-2.709264,1.936423,-1.363935,-4.321434,-6.367614,-0.650000,3.844424,-2.344844,0.426418,-1.820167,-5.890534,-1.076403,4.201090,-1.111580,2.415288,-0.096227,2.788463
2246,-15.691800,5.857986,-2.309148,-2.420708,-0.657822,-2.666565,0.722860,-1.412569,-0.949756,-1.418549,0.600040,-0.934870,0.714662,-0.873874,-0.378437,-0.468672,-0.844780,-1.313804,1.917636,1.157015
3062,-14.561600,-0.697327,0.632804,-1.909114,-4.894620,-0.862991,-0.717960,-1.097901,1.462603,-2.028652,-0.195036,1.967712,0.955920,0.830309,-0.450738,-0.857832,-1.488830,0.440262,-0.362554,0.029232
3490,-15.152856,1.996056,-0.526772,-2.423586,-3.561816,-2.179371,0.661697,0.442748,-0.312896,-0.262648,1.597340,0.870975,0.133378,0.084501,0.566362,1.118104,-1.029428,1.603187,-0.810497,-1.056735


In [98]:
model = Sequential([
            Input((x_train.shape[-1],)),
            Dense(2, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        
# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'],)

checkpoint = ModelCheckpoint(
    'best_model_is_dynamic.keras',          # 저장할 파일명
    monitor='val_accuracy',   # 모니터링할 지표
    save_best_only=True,      # 가장 좋은 성능의 모델만 저장
    mode='max',               # 최대값을 찾기 때문에 'max'
    verbose=1                # 저장 시 메시지 출력
)
 # 모델 학습
result = model.fit(x_train, y_train,shuffle=True,epochs=50,validation_split=.2,callbacks=[checkpoint]).history

Epoch 1/50
110/118 [==========================>...] - ETA: 0s - loss: 1.2139 - accuracy: 0.2625
Epoch 1: val_accuracy improved from -inf to 0.43358, saving model to best_model.keras
118/118 [==============================] - 1s 4ms/step - loss: 1.1874 - accuracy: 0.2737 - val_loss: 0.8035 - val_accuracy: 0.4336
Epoch 2/50
 81/118 [===================>..........] - ETA: 0s - loss: 0.6726 - accuracy: 0.5741
Epoch 2: val_accuracy improved from 0.43358 to 0.74920, saving model to best_model.keras
118/118 [==============================] - 0s 2ms/step - loss: 0.6276 - accuracy: 0.6202 - val_loss: 0.4824 - val_accuracy: 0.7492
Epoch 3/50
117/118 [============================>.] - ETA: 0s - loss: 0.3961 - accuracy: 0.8088
Epoch 3: val_accuracy improved from 0.74920 to 0.83847, saving model to best_model.keras
118/118 [==============================] - 0s 2ms/step - loss: 0.3958 - accuracy: 0.8087 - val_loss: 0.3332 - val_accuracy: 0.8385
Epoch 4/50
114/118 [===========================>..] - E

# SMOTE 기법 전처리 적용해서 